<a href="https://colab.research.google.com/github/Shopping-Yuan/ML2021HW/blob/Shopping_vscode_branch/HW5/HW05_modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

------
###Part 0 setting and installing package
------
###Part 1 preparing data set
------
######load data file
######clean data
######pick up line pairs
######tokenize : using sentencepiece
######make data set
------
###Part 2 make model
------
######positional encoding layer
######multihead attention layer
######encoder layer(s)
######decoder layer(s)
######transformer layer
------
###Part 3 training and validation process
------
######Noam optimizer
######label smoothing
######beam search
######bleu
######training and validation function
######main function
######get plot
------
###Part 4 inference
------
######infer dataset
######infer function
######inference main function


setting
======
>Here are all parameters using in this project.

In [1]:
setting = {
# information of the path of dataset
"data_info" : {
    "drive_path":"/content/drive",
    "document":"/content/drive/MyDrive",
    "raw_file_name":"/ted2020.tgz",
    "unzip_path":"/train_dev/",
    "source":{
        "lang":"en",
        "raw_data_path":"/train_dev/raw.en",
        "clean_data_path":"/train_dev/clean_en.txt",
        "tokenized_train_data":"/train_dev/tokenized_train_data_en.txt",
        "tokenized_val_data":"/train_dev/tokenized_val_data_en.txt"
        },
    "target":{
        "lang":"zh",
        "raw_data_path":"/train_dev/raw.zh",
        "clean_data_path":"/train_dev/clean_zh.txt",
        "tokenized_train_data":"/train_dev/tokenized_train_data_zh.txt",
        "tokenized_val_data":"/train_dev/tokenized_val_data_zh.txt"
        }
},
# tokenized setting for spm
"tokenized_setting" : {
    "vocab_size" : 8000,
    "character_coverage" : 1,
    "model_type" : "bpe", # "unigram",
    "input_sentence_size" : 400000,
    "shuffle_input_sentence" : True,
    "normalization_rule_name" : "nmt_nfkc_cf",
    "pad_id":0,
    "unk_id":1,
    "bos_id":2,
    "eos_id":3,
    "max_l":400
},
# model structure setting
"model" : {
      "encoder_embedding_dimension" : 256,
      "decoder_embedding_dimension" : 256,
      "feedforward_dimension" : 2048,
      "num_heads" : 2,
      "dropout_p" : 0.0,
      "layer_num" : 6
},

# setting in training and validation process ,
# including optimization setting.
"training_hparas" : {
    "total_step" : 60000,
    "do_valid_step" : 60000,
    "early_stop_step" : 2,
    "temp_save_step" : 1000,
    "train_batch_size" : 40,
    "valid_batch_size" : 100,
    "workers" : 2,
    "label_smoothing" : 0.1,
    "beam_num" : 2,
    "optimization":{
        "factor" : 2,
        "warmup"  : 4000,
        "optimizer" : {
                "lr" : 0,
                "betas" : (0.9, 0.98),
                "eps" : 1e-9,
                "weight_decay" : 0.0001
                }
            },
    "model_saving_path" : "/content/drive/MyDrive/model.pth",
    "model_temporary_saving_path" : "/content/drive/MyDrive/temporary_model.pth"
},
"inference_out_path" : "/infer_out.json"
}


installing package
------

In [2]:
# used in part 1
!pip install sentencepiece
# used in part 1 and 3
!pip install tqdm
# used in part 2
!pip install torchinfo
# used in part 3
!pip install torcheval

preparing data set
=============

load data file
-------------
>Here I load dataset from my drive,  
>but it also can be download from the link below.

In [3]:
# step 1 : download dataset from drive to google colab
# original dataset is in "https://mega.nz/#!vEcTCISJ!3Rw0eHTZWPpdHBTbQEqBDikDEdFPr7fI8WxaXK9yZ9U"

path_doc = setting["data_info"]["document"]
rawdata_file_name = setting["data_info"]["raw_file_name"]
rawdata_file_path = path_doc + rawdata_file_name
unzip_path = path_doc + setting["data_info"]["unzip_path"]

# mount drive
from google.colab import drive
drive_path = setting["data_info"]["drive_path"]
drive.mount(drive_path)

# copy file from drive
# import shutil
# shutil.copyfile(path_doc + rawdata_file_name, rawdata_file_path)

# step 2 : unzip dataset
import tarfile
# open file
file = tarfile.open(rawdata_file_path)
# extracting file
file.extractall(unzip_path)
file.close()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


clean data
------
>First each dataset (source or target) is clean  
>seperately, change to halfwidth and remove/replace  
>some kind of punctuations.

>Also because the number of sentences in one line may be  
>different in line pairs of source and target set (its an error),  
>some special punctuations is add to the end of sentences  
>for the next process dealing with these problem by  
>using sentence pairs instead of lines pairs to form datasets.



In [4]:
import unicodedata
import string
import re
# convert fullwidth to halfwidth
def to_halfwidth(string):
  return "".join(unicodedata.normalize('NFKC',letter) for letter in string)
def clean_s_zh(s):
    s = to_halfwidth(s)
    # step 1 : delete — _
    delete = " _()[]"
    delete_rules = s.maketrans("","",delete)
    s = s.translate(delete_rules)

    # step 2 : replace “” with ""
    to_be_replace = '“”'
    replace = '""'
    replace_dict = dict(zip(to_be_replace,replace))

    # step 3 : add **END** before and after punctuation

    """
    The number of sentences in one line may be different
    in line pairs of source and target set.
    "**END**" is add after "。!?" and ".!?", which can be used
    to check if the number of sentence in the pair are equal
    in the next process.
    also in english, "." may be use in abbreviation,
    these different use must be identified.

    """

    punctuation = "。!?"
    for char in punctuation:
      replace_dict[char] = char + "**END**"

    replace_rules = s.maketrans(replace_dict)
    s = s.translate(replace_rules)

    zh_list = s.strip("\n").split("\n")

    return zh_list

def clean_s_en(s):
    s = to_halfwidth(s)

    replace_dict = {}

    delete = "-()[]"
    for char in delete:
      replace_dict[char] = ""

    punctuation = "!?"
    for char in punctuation:
      replace_dict[char] = char + "**END**"
    replace_rules = s.maketrans(replace_dict)
    s = s.translate(replace_rules)

    # Identify if "." is used in abbreviation,
    # if not, add "**END**" after it.
    pattern = re.compile(r"(?<!([.\s\r\n\f][a-zA-Z]))[.]")
    s = pattern.sub("**END**",s)

    # test pattern
    # pattern = re.compile(r"(?<!([.\s\r\n\f][a-zA-Z]))[.]")
    # result = pattern.sub("**END**","There are many people in U.S. w.r.t. in Taiwan.Thank you.")

    en_list = s.strip("\n").split("\n")

    return en_list

pick up line pairs
------
>pick up line pairs has equal number of sentences and  
>split them into sentences to form sourse/target dataset.  
>Remove sentences with too many words for training and validation.

In [5]:
# using "**END**" to split line pairs to check if they have equal sentence
def divide_by_END(s):
    list_s = []
    for line_string in s.strip("**END**").split("**END**"):
      if line_string not in [""," "]:
         list_s.append(line_string)
    return(list_s)
'''
warning : devide_en_again function is apply just beacause
in "this" dataset english sentences end with ":" or ";"
sometimes not splited well.
If the dataset is change, this part may need to be
eliminated or modified.
'''
def devide_en_again(s,punctuation = ":;"):
    replace_dict = {}
    for char in punctuation:
      replace_dict[char] = char + "**END**"

    replace_rules_src = s.maketrans(replace_dict)
    new_s = divide_by_END(s.translate(replace_rules_src))
    return new_s

# remove "sentence" if it is too long.
def remove_too_long(src_list,tgt_list,threshold):
    too_long_src = 0
    too_long_tgt = 0
    remove = False
    new_s = []
    new_t = []
    for i in range(len(src_list)):
      if ((len(src_list[i])>threshold)):
        remove = True
        too_long_src += 1
      if (len(tgt_list[i])>threshold):
        remove = True
        too_long_tgt += 1
      if remove == False:
        new_s.append(src_list[i])
        new_t.append(tgt_list[i])
      else :
        remove = False
    return(new_s,new_t,too_long_src,too_long_tgt)

# pick up good line pairs for traning and validation model
def check_data_pairs(src_list,tgt_list,threshold):
    index = 0
    new_src_list = []
    new_tgt_list = []

    same = 0
    add_next = 0
    split_again = 0
    not_use = 0

    while(index < len(src_list)):

      src = divide_by_END(src_list[index])
      tgt = divide_by_END(tgt_list[index])

      # case 1 : src is as long as tgt , finished.
      if len(src) == len(tgt):
        new_src_list += src
        new_tgt_list += tgt
        same += 1
        index += 1

      else :
        # if it is not the last one : both src and tgt add next sentence
        if index != len(src_list)-1:
          src_add_next = divide_by_END(src_list[index] + src_list[index+1])
          tgt_add_next = divide_by_END(tgt_list[index] + tgt_list[index+1])
          # case 2 : src_add_next is as long as tgt_add_next , finished.
          if len(src_add_next) == len(tgt_add_next):
            new_src_list += src_add_next
            new_tgt_list += tgt_add_next
            add_next += 2
            index += 2

          # using new punctuation to divide tgt (english) sentence.
          # note that this part could cause negative effects if the dataset is change.
          else :
            src_add_next = devide_en_again(src_list[index] + src_list[index+1])
            # case 3 : src_add_next is as long as tgt_add_next , finished.
            if len(src_add_next) == len(tgt_add_next):
              new_src_list += src_add_next
              new_tgt_list += tgt_add_next
              split_again +=2
              index += 2

            # case 4 : sentence will not be used.
            else :
              not_use += 1
              # if to_do == 1 :
              #   print(index,src_add_next,tgt_add_next,len(src_add_next),len(tgt_add_next))
              index += 1

        # if it is the last one
        else :
          not_use += 1
          index += 1
    # print information
    print(f"The original total number of line is {index}.")
    print(f"The number of line pairs have the equal sentences is {same}.")
    print(f"The number of line pairs have the equal sentences after combine the next lines is {add_next}.")
    print(f"The number of line pairs have the equal sentences after combine the next lines"+\
       f"and resplit english lines using :; is {split_again}.")
    print(f"The number of line we don't use is {not_use}.")
    print(f"Note that {index} = {same}+{add_next}+{split_again}+{not_use}.")

    # remove long lines
    print(f"The total number of sentence pairs before remove long sentences is {len(new_src_list)}.")
    new_src_list,new_tgt_list,too_long_src,too_long_tgt = remove_too_long(new_src_list,new_tgt_list,threshold)
    print(f"The finally total number of sentence pairs using is {len(new_src_list)}.")
    print(f"Note that {len(new_src_list)} are the number of sentence pairs, not line pairs")

    return(new_src_list,new_tgt_list)

load and clean data
------

In [6]:
# load and clean data
def load_file(path,function):
    with open(path, "r") as f:
      data = f.read()
      return function(data)
# saving to new path
def clean_data_and_save(
    path_doc,raw_src_path,raw_tgt_path,
    clean_src_path,clean_tgt_path,threshold
    ):
    raw_src_path = path_doc + raw_src_path
    raw_tgt_path = path_doc + raw_tgt_path
    src = load_file(raw_src_path,clean_s_en),
    tgt = load_file(raw_tgt_path,clean_s_zh),
    # src , tgt are tuples with only one term : src_list, tgt_list
    src_list = src[0]
    tgt_list = tgt[0]
    clean_src_list, clean_tgt_list = check_data_pairs(src_list,tgt_list,threshold)
    with open(path_doc + clean_src_path, "w") as f:
      f.write("\n".join(clean_src_list))
    with open(path_doc + clean_tgt_path, "w") as f:
      f.write("\n".join(clean_tgt_list))
# test clean_data_and_save
# clean_data_and_save(
#     path_doc = setting["data_info"]["document"],
#     raw_src_path = setting["data_info"]["source"]["raw_data_path"],
#     raw_tgt_path = setting["data_info"]["target"]["raw_data_path"],
#     clean_src_path = setting["data_info"]["source"]["clean_data_path"],
#     clean_tgt_path = setting["data_info"]["target"]["clean_data_path"],
#     threshold = setting["tokenized_setting"]["max_l"]
# )

tokenize
------
>using sentencepiece to tokenize sentences,  
>first make the english/chinese dictionary separately,  
>then use these dict to encode sentence pair in dataset,  
>including add bos/eos/padding to tokenized sentences.  
>Finally split then into train/val set and save.

In [7]:
import sentencepiece as spm
import numpy as np
from tqdm import tqdm
import torch.utils.data as data
def tokenized(path_doc,
       clean_data_path,
       vocab_size,
       lang,
       tokenized_setting
       ):
  model_prefix = f"spm_{vocab_size}_{lang}"
  spm.SentencePieceTrainer.train(
      input= path_doc+clean_data_path,
      **tokenized_setting,
      model_prefix = path_doc +"/"+ model_prefix,
  )
  return(model_prefix)

def get_tokenizers(path_doc,vocab_size,src_lang,tgt_lang):
  src_tokenizer = spm.SentencePieceProcessor(model_file = path_doc + f"/spm_{vocab_size}_{src_lang}" +".model")
  tgt_tokenizer = spm.SentencePieceProcessor(model_file = path_doc + f"/spm_{vocab_size}_{tgt_lang}" +".model")
  return src_tokenizer,tgt_tokenizer

def bos_eos_padding(dataset,
          max_l,
          src_tokenizer,
          tgt_tokenizer,
          bos_id,
          eos_id,
          pad_id,
          ):


  padding_src = []
  padding_tgt = []
  len_s = 0
  len_t = 0
  for src,tgt in dataset:
    s = src_tokenizer.encode(src, out_type=int)
    s = np.append(s,[eos_id])
    s = np.append([bos_id],np.pad(s,(0, max_l-len(s)-1), constant_values = pad_id))
    padding_src.append(s)

    t = tgt_tokenizer.encode(tgt, out_type=int)
    t = np.append(t,[eos_id])
    t = np.append([bos_id],np.pad(t,(0, max_l-len(t)-1), constant_values = pad_id))
    padding_tgt.append(t)

  return(list(zip(padding_src,padding_tgt)))
# test SentencePieceProcessor and bos_eos_padding
# s_src = spm.SentencePieceProcessor(model_file="/content/spm8000_en.model")
# s_src.encode("hello world!", out_type=int)
# bos_eos_padding([("hello world","_哈囉")],5,10)

def data_set_preparing(path_doc,
            clean_src_path,
            clean_tgt_path,
            max_l,
            src_tokenizer,
            tgt_tokenizer,
            st_train_path,
            st_val_path,
            tt_train_path,
            tt_val_path,
            bos_id,
            eos_id,
            pad_id,
            ):
    src_set = []
    tgt_set = []

    with open(path_doc+clean_src_path,"r") as in_f :
      for line in tqdm(in_f):
        src_set.append(line)
    with open(path_doc+clean_tgt_path,"r") as in_f :
      for line in tqdm(in_f):
        tgt_set.append(line)

    dataset = list(zip(src_set,tgt_set))
    dataset = bos_eos_padding(dataset,max_l,src_tokenizer,tgt_tokenizer)
    train_set, valid_set = data.random_split(dataset,[0.99,0.01])
    # print(train_set[0][0])

    with open(path_doc + st_train_path, 'w') as out_f:
      for line_pair in tqdm(train_set):
        out_f.write(" ".join(str(x) for x in line_pair[0])+"\n")
    with open(path_doc + st_val_path, 'w') as out_f:
      for line_pair in tqdm(valid_set):
        out_f.write(" ".join(str(x) for x in line_pair[0])+"\n")
    with open(path_doc + tt_train_path, 'w') as out_f:
      for line_pair in tqdm(train_set):
        out_f.write(" ".join(str(x) for x in line_pair[1])+"\n")
    with open(path_doc + tt_val_path, 'w') as out_f:
      for line_pair in tqdm(valid_set):
        out_f.write(" ".join(str(x) for x in line_pair[1])+"\n")

In [8]:
def tokenized_data(vocab_size,tokenized_setting,max_l,path_doc,clean_src_path,
          clean_tgt_path,src_lang,tgt_lang,st_train_path,st_val_path,
          tt_train_path,tt_val_path,bos_id,eos_id,pad_id):
  tokenized(path_doc,clean_src_path,vocab_size,src_lang,tokenized_setting)
  tokenized(path_doc,clean_tgt_path,vocab_size,tgt_lang,tokenized_setting)
  src_tokenizer,tgt_tokenizer = get_tokenizers(path_doc,vocab_size,src_lang,tgt_lang)
  data_set_preparing(path_doc,clean_src_path,clean_tgt_path,max_l,src_tokenizer,
           tgt_tokenizer,st_train_path,st_val_path,tt_train_path,tt_val_path,
           bos_id,eos_id,pad_id)
  return src_tokenizer,tgt_tokenizer

# test tokenized_data()
# src_tokenizer,tgt_tokenizer = tokenized_data(
#     vocab_size = setting["tokenized_setting"]["vocab_size"],
#     tokenized_setting = {k:setting["tokenized_setting"][k] for k in \
#               set(list(setting["tokenized_setting"].keys()))-{"vocab_size","max_l"}},
#     max_l = setting["tokenized_setting"]["max_l"],
#     path_doc = setting["data_info"]["document"],
#     clean_src_path = setting["data_info"]["source"]["clean_data_path"],
#     clean_tgt_path = setting["data_info"]["target"]["clean_data_path"],
#     src_lang = setting["data_info"]["source"]["lang"],
#     tgt_lang = setting["data_info"]["target"]["lang"],
#     st_train_path = setting["data_info"]["source"]["tokenized_train_data"],
#     st_val_path = setting["data_info"]["source"]["tokenized_val_data"],
#     tt_train_path = setting["data_info"]["target"]["tokenized_train_data"],
#     tt_val_path = setting["data_info"]["target"]["tokenized_val_data"])

make data set
------
> Using tokenized data to make dataset.  
> Classmethod : padding_mask_batch which  
> where the key padding mask is constucted  
> also defined here.

In [9]:
import torch
from tqdm import tqdm
import numpy as np
from torch.utils.data import Dataset

class myDataset(Dataset):
  def __init__(self,src_path,tgt_path):

    self.src_path = src_path
    self.tgt_path = tgt_path

    src_list = []
    with open(self.src_path,"r") as f :
      d_l = f.readlines()
      for line in tqdm(d_l):
        int_list = [int(i) for i in line.split()]
        src_list.append(int_list)
    self.src = torch.LongTensor(src_list)

    tgt_list = []
    with open(self.tgt_path,"r") as f :
      l_l = f.readlines()
      for line in tqdm(l_l):
        int_list = [int(i) for i in line.split()]
        tgt_list.append(int_list)
    self.tgt = torch.LongTensor(tgt_list)

  def __len__(self):
    return len(self.src)

  def __getitem__(self, index):
    return self.src[index], self.tgt[index]

  # make key padding mask
  @classmethod
  def padding_mask_batch(cls,batch,pad_id):
    """Collate a batch of data."""
    src, tgt = zip(*batch)
    src = torch.stack(src)
    tgt = torch.stack(tgt)
    src_padding = (src == pad_id)
    tgt_padding = (tgt == pad_id)

    return src, tgt , src_padding, tgt_padding
# test myDataset
# data = []
# with open("/content/train_dev/tokenized_train_data_en.txt","r") as f :
#   d_l = f.readlines()
#   for line in tqdm(d_l):
#     int_list = [int(i) for i in line.split()]
#     data.append(int_list)
# print(data[0])

In [10]:
from torch.utils.data import DataLoader
import gc
def get_data_set(train_batch_size,valid_batch_size,num_workers,path_doc,
         st_train_path,st_val_path,tt_train_path,tt_val_path,pad_id):

  train_set = myDataset(src_path = path_doc + st_train_path,
              tgt_path = path_doc + tt_train_path,
              )
  valid_set = myDataset(src_path = path_doc + st_val_path,
              tgt_path = path_doc + tt_val_path,
              )
  train_loader = DataLoader(
    train_set,
    batch_size = train_batch_size,
    shuffle = True,
    num_workers = num_workers,
    pin_memory = True,
    collate_fn = lambda x : myDataset.padding_mask_batch(x,
                   pad_id = pad_id)
  )
  valid_loader = DataLoader(
    valid_set,
    batch_size = valid_batch_size,
    num_workers = num_workers,
    pin_memory = True,
    collate_fn = lambda x : myDataset.padding_mask_batch(x,
                   pad_id = pad_id)
  )
  del train_set,valid_set
  gc.collect()
  return train_loader,valid_loader
# test get_data_set()
# train_set,valid_set = get_data_set()
# batch = next(iter(valid_set))
# src,tgt,src_mask,tgt_mask = batch
# print(src.shape)

make model
======
positional encoding layer
------
>The first layer is embedding layer, where each integers  
>in encoder sentence will be represent by a vector.   
>I use build-in class in pytorch to finish these part,    
>and combine it with encoder layers to form my encoder.

>The layer below is the second layer :positional encoding layer  
>in this layer the position infomation is add to each "word"  
>in the sentence.
>Here I use parameters instead of constant as  
>position infomation so they will change during training process.

In [11]:
import torch
import torch.nn as nn
class Positional_Encoding(nn.Module):
    def __init__(self,max_sentence_length,embedding_dimension):
      super().__init__()
      self.dropout = nn.Dropout(0.1)
      self.encoding_values = nn.Parameter(nn.init.normal_(torch.empty(max_sentence_length,1, embedding_dimension)))
    def forward(self, x):
        # the shape of x : [batch,length,e_dim]
        # the shape of self.encoding_values : [batch,length,e_dim]
        x = x + self.encoding_values.unsqueeze(0)
        x = x.squeeze(-2)
        return self.dropout(x)

multihead attention layer
------


In [12]:
import torch.nn.functional as F
import torchvision
import math
from torchinfo import summary
# This part is modify from pytorch : torch.nn.functional.scaled_dot_product_attention
# Efficient implementation equivalent to the following:
class Scaled_Dot_Product_Attention(nn.Module):
    def __init__(self,max_sentence_length,dropout_p):
      super().__init__()
      self.dropout_p = dropout_p
      self.max_l = max_sentence_length
      attn_bias = torch.zeros(self.max_l, self.max_l)
      temp_mask = torch.ones(self.max_l, self.max_l, dtype=torch.bool).tril(diagonal=0)
      attn_bias = attn_bias.masked_fill_(temp_mask.logical_not(), float("-inf"))
      self.register_buffer("attn_bias",attn_bias)

    def forward(self, is_last_batch, query, key, value, padding_mask=None, is_causal=False, scale=None) -> torch.Tensor:
      # Efficient implementation equivalent to the following:

      scale_factor = 1 / math.sqrt(query.size(-1)) if scale is None else scale
      attn_weight = query @ key.transpose(-2, -1) * scale_factor

      if is_causal:
        if is_last_batch:
          self.attn_bias = self.attn_bias[:query.size(-2),:query.size(-2)]
        self.attn_bias.to(query.dtype)
        attn_weight += self.attn_bias

      if padding_mask is not None:
          if padding_mask.dtype == torch.bool:
            padding_mask = torch.zeros_like(padding_mask,dtype = float).masked_fill_(padding_mask, (float("-inf")))

          padding_mask = padding_mask.unsqueeze(0).unsqueeze(0)
          padding_mask.to(query.dtype)

          attn_weight = attn_weight.transpose(-4,-2)
          attn_weight += padding_mask
          attn_weight = attn_weight.transpose(-4,-2)

      attn_weight = torch.softmax(attn_weight, dim=-1)
      attn_weight = torch.dropout(attn_weight, self.dropout_p, train=True)
      return attn_weight @ value
# test scaled_dot_product_attention
# t = torch.rand([2,3,4,5])
# mask = torch.tensor([[False,False,True,True],[False,True,False,True]],dtype = torch.bool)
# print(scaled_dot_product_attention("cpu",t,t,t,padding_mask= mask, is_causal=True))
# from torch.nn.functional import scaled_dot_product_attention
class My_MultiHeadedAttention(nn.Module):
    def __init__(self, max_sentence_length, kv_input_dimension, embedding_dimension, num_heads, dropout_p, if_decoder = False):
        '''
        embedding_dimension = input dimension
        note that there are residual sublayers in MultiHeadedAttention
        '''
        super().__init__()
        assert embedding_dimension % num_heads == 0, "embed_dim must be divisible by num_heads"
        self.max_l = max_sentence_length
        self.kv_d = kv_input_dimension
        self.d = embedding_dimension
        self.num_heads = num_heads
        self.dropout_p = dropout_p
        self.is_causal = if_decoder
        self.sdpa = Scaled_Dot_Product_Attention(self.max_l,self.dropout_p)
        self.linear_for_q = nn.Linear(self.d, self.d)
        self.linear_for_kv = nn.Linear(self.kv_d, 2 * self.d)
        self.linear_out_project = nn.Linear(self.d, self.d)

    def forward(self, is_last_batch, q_input_data, kv_input_data , padding_mask = None):

        query = self.linear_for_q(q_input_data)
        key, value = self.linear_for_kv(kv_input_data).split(self.d,dim = -1)

        query,key,value = \
          map(lambda x : x.view(x.size(0),x.size(1),self.num_heads,self.d//self.num_heads),[query,key,value])
        query,key,value = \
          map(lambda x : x.transpose(-2,-3),[query,key,value])

        x = self.sdpa(is_last_batch,query,key,value,padding_mask = padding_mask,is_causal = self.is_causal)
        x = x.transpose(-2,-3).contiguous()
        x = x.view(x.size(0),x.size(1),self.d)
        x = self.linear_out_project(x)

        return x
# test My_MultiHeadedAttention
# model = My_MultiHeadedAttention(64,128,2,0.0)
# q_input = torch.rand(32,400,128)
# kv_input = torch.rand(32,400,64)
# mask = (torch.FloatTensor(32,400).uniform_() > 0.8)
# print(model(q_input,kv_input,mask).size())
# print(summary(model,device = "cpu",q_input_data = q_input, kv_input_data = kv_input,padding_mask = mask))

encoder layer(s)
------

In [13]:
import math
class My_Encoder_Layer(nn.Module):
  def __init__(self,max_sentence_length,embedding_dimension,feedforward_dimension,num_heads,dropout_p):
    super().__init__()
    self.max_l = max_sentence_length
    self.emb_dim = embedding_dimension
    self.fwd_dim = feedforward_dimension
    self.num_heads = num_heads
    self.dropout_p = dropout_p

    self.attention = My_MultiHeadedAttention(self.max_l, self.emb_dim, self.emb_dim, self.num_heads, self.dropout_p)
    self.layer_norm_attn = nn.LayerNorm(self.emb_dim)
    self.drop_out_attn_layernorm = nn.Dropout(self.dropout_p)

    self.feedforward = nn.Sequential(
    nn.Linear(self.emb_dim,self.fwd_dim),
    nn.ReLU(),
    nn.Linear(self.fwd_dim,self.emb_dim)
    )
    self.layer_norm_feedforward = nn.LayerNorm(self.emb_dim)
    self.drop_out_feedforward_layernorm = nn.Dropout(self.dropout_p)


  def forward(self,is_last_batch,x,padding_mask):
    x = x + self.attention(is_last_batch,x,x,padding_mask)
    x = self.layer_norm_attn(x)

    x = self.drop_out_attn_layernorm(x)

    x = x + self.feedforward(x)
    x = self.layer_norm_feedforward(x)
    x = self.drop_out_feedforward_layernorm(x)

    return x
# test My_Encoder_Layer
# model = My_Encoder_Layer("cpu",128,256,2,0.0)
# input = torch.rand((32,400,128))
# mask = (torch.FloatTensor(32,400).uniform_() > 0.8)
# print(model(input,mask).size())
# print(summary(model,input_data = input,padding_mask = mask))
# print(model.state_dict().keys())
class My_Encoder(nn.Module):
  def __init__(self,max_sentence_length,dictionary_length,embedding_dimension,feedforward_dimension,
         padding_idx, num_heads, dropout_p, layer_num):
    super().__init__()
    self.max_l = max_sentence_length
    self.dict_l = dictionary_length
    self.emb_dim = embedding_dimension
    self.fwd_dim = feedforward_dimension
    self.padding_idx = padding_idx
    self.num_heads = num_heads
    self.dropout_p = dropout_p
    self.layer_num = layer_num

    self.encoder_embedding = nn.Embedding(self.dict_l,self.emb_dim,self.padding_idx)
    self.positional_encoding = Positional_Encoding(self.max_l,self.emb_dim)
    self.encoder = nn.ModuleList([My_Encoder_Layer(self.max_l,self.emb_dim,self.fwd_dim,\
                    self.num_heads,self.dropout_p) for i in range(layer_num)])

  def forward(self,is_last_batch,input,padding_mask):
    x = self.encoder_embedding(input.unsqueeze(-1))* math.sqrt(self.emb_dim)
    x = self.positional_encoding(x)

    for index,module in enumerate(self.encoder):
      if index == 0:
        x = module(is_last_batch,x,padding_mask)
      else:
        x = module(is_last_batch,x,None)
    return x
# test My_Encoder
# model = My_Encoder("cpu",400,8000,128,256,0,2,0.0,2)
# input = torch.randint(0,7999,(32,400),dtype = torch.long)
# mask = (torch.FloatTensor(32,400).uniform_() > 0.8)
# print(model(input,mask).size())
# print(summary(model,input_data = input,padding_mask = mask))
# print(model.state_dict().keys())

decoder layer(s)
------

In [14]:
import math
class My_Decoder_Layer(nn.Module):
  def __init__(self,max_sentence_length,encoder_embedding_dimension,embedding_dimension,feedforward_dimension,num_heads,dropout_p):
    super().__init__()
    self.max_l = max_sentence_length
    self.encoder_dim = encoder_embedding_dimension
    self.emb_dim = embedding_dimension
    self.fwd_dim = feedforward_dimension
    self.num_heads = num_heads
    self.dropout_p = dropout_p

    self.self_attention = My_MultiHeadedAttention \
     (self.max_l,self.emb_dim,self.emb_dim, num_heads = self.num_heads,\
     dropout_p = self.dropout_p, if_decoder = True)
    self.layer_norm_sa = nn.LayerNorm(self.emb_dim)
    self.drop_out_sa = nn.Dropout(0)

    self.feedforward_sa = nn.Sequential(
    nn.Linear(self.emb_dim,self.fwd_dim),
    nn.ReLU(),
    nn.Linear(self.fwd_dim,self.emb_dim)
    )
    self.layer_norm_sa_fw = nn.LayerNorm(self.emb_dim)
    self.drop_out_sa_fw = nn.Dropout(0)

    self.cross_attention = My_MultiHeadedAttention \
    (self.max_l,self.encoder_dim, self.emb_dim, num_heads = self.num_heads,
    dropout_p = self.dropout_p, if_decoder = True)
    self.layer_norm_ca = nn.LayerNorm(self.emb_dim)
    self.drop_out_ca = nn.Dropout(0)

    self.feedforward_ca = nn.Sequential(
    nn.Linear(self.emb_dim,self.fwd_dim),
    nn.ReLU(),
    nn.Linear(self.fwd_dim,self.emb_dim)
    )
    self.layer_norm_ca_fw = nn.LayerNorm(self.emb_dim)
    self.drop_out_ca_fw = nn.Dropout(0)

  def forward(self,is_last_batch,encoder_input,input,padding_mask):

    x = input + self.self_attention(is_last_batch,input,input,padding_mask)
    x = self.layer_norm_sa(x)
    x = self.drop_out_sa(x)

    x = x + self.feedforward_sa(x)
    x = self.layer_norm_sa_fw(x)
    x = self.drop_out_sa_fw(x)

    x = x + self.cross_attention(is_last_batch,x,encoder_input,padding_mask)
    x = self.layer_norm_ca(x)
    x = self.drop_out_ca(x)

    x = x + self.feedforward_ca(x)
    x = self.layer_norm_ca_fw(x)
    x = self.drop_out_ca_fw(x)

    return x
class My_Decoder(nn.Module):
  def __init__(self,max_sentence_length, dictionary_length, encoder_embedding_dimension,
         embedding_dimension, feedforward_dimension, padding_idx, num_heads, dropout_p, layer_num):
    super().__init__()
    self.max_l = max_sentence_length
    self.dict_l = dictionary_length
    self.encoder_dim = encoder_embedding_dimension
    self.emb_dim = embedding_dimension
    self.fwd_dim = feedforward_dimension
    self.padding_idx = padding_idx
    self.num_heads = num_heads
    self.dropout_p = dropout_p
    self.layer_num = layer_num

    self.decoder_embedding = nn.Embedding(self.dict_l,self.emb_dim,padding_idx=self.padding_idx)
    self.positional_encoding = Positional_Encoding(self.max_l,self.emb_dim)
    self.decoder = nn.ModuleList([My_Decoder_Layer(self.max_l,self.encoder_dim,self.emb_dim,\
                    self.fwd_dim,self.num_heads,self.dropout_p) for i in range(self.layer_num)])
    # self.encoder = My_Encoder_Layer(self.emb_dim,self.fwd_dim)

    self.generator = nn.Linear(self.emb_dim,self.dict_l)

  def forward(self,is_last_batch,encoder_input,input,padding_mask):
    x = self.decoder_embedding(input.unsqueeze(-1))* math.sqrt(self.emb_dim)
    x = self.positional_encoding(x)
    # x = self.encoder(x,padding_mask)
    for index,module in enumerate(self.decoder):
      if index == 0:
        x = module(is_last_batch,encoder_input,x,padding_mask)
      else:
        x = module(is_last_batch,encoder_input,x,None)
    x = self.generator(x)
    x = F.log_softmax(x,dim = -1)
    return x
# test My_Decoder
# model = My_Decoder("cpu",400,8000,128,64,256,0,2,0.0,2)
# encoder_input = torch.rand(32,400,128)
# input = torch.randint(0,7999,(32,400),dtype = torch.long)
# mask = (torch.FloatTensor(32,400).uniform_() > 0.8)
# print(model(encoder_input = encoder_input,input = input, padding_mask = mask).size())
# print(summary(model,encoder_input = encoder_input,input = input, padding_mask = mask))
# print(model.state_dict().keys())

transformer layer
------

In [15]:
class My_Transformer(nn.Module):
  def __init__(self,max_sentence_length,dictionary_length,padding_idx,
         encoder_embedding_dimension,decoder_embedding_dimension,
         feedforward_dimension,num_heads,dropout_p,layer_num):
    super().__init__()
    self.max_l = max_sentence_length
    self.dict_l = dictionary_length
    self.padding_idx = padding_idx
    self.en_dim = encoder_embedding_dimension
    self.de_dim = decoder_embedding_dimension
    self.fw_dim = feedforward_dimension
    self.num_heads = num_heads
    self.dropout_p = dropout_p
    self.layer_num = layer_num
    self.encoder = My_Encoder \
     (self.max_l,self.dict_l,self.en_dim,self.fw_dim,
      self.padding_idx,self.num_heads,self.dropout_p,self.layer_num)
    self.decoder = My_Decoder \
     (self.max_l,self.dict_l,self.en_dim,self.de_dim,self.fw_dim,
      self.padding_idx,self.num_heads,self.dropout_p,self.layer_num)

  def forward(self,is_last_batch,src,tgt,src_mask,tgt_mask):
    memory = self.encoder(is_last_batch,src,src_mask)
    outputs = self.decoder(is_last_batch,memory,tgt,tgt_mask)
    return outputs

def build_model(max_sentence_length,dictionary_length,padding_idx,encoder_embedding_dimension,
         decoder_embedding_dimension,feedforward_dimension,num_heads,dropout_p,layer_num):
  return My_Transformer(max_sentence_length,dictionary_length,padding_idx,
              encoder_embedding_dimension,decoder_embedding_dimension,
              feedforward_dimension,num_heads,dropout_p,layer_num)
# test Transformer
# model = My_Transformer(max_sentence_length = 12,
#             dictionary_length = 15,
#             padding_idx = 0,
#             encoder_embedding_dimension = 128,
#             decoder_embedding_dimension = 64,
#             feedforward_dimension = 256,
#             num_heads = 2,
#             dropout_p = 0,
#             layer_num = 2)
# src = torch.randint(0,15,(5,12),dtype = torch.long)
# tgt = torch.randint(0,15,(5,12),dtype = torch.long)
# src_mask = torch.cat(((torch.FloatTensor(5,6).uniform_() > 1),(torch.FloatTensor(5,6).uniform_() > 0.15)),dim =1)
# tgt_mask = torch.cat(((torch.FloatTensor(5,4).uniform_() > 1),(torch.FloatTensor(5,8).uniform_() > 0.15)),dim =1)
# out = model(False,src,tgt,src_mask,tgt_mask)
# print(summary(model,src = src,tgt = tgt,src_mask = src_mask,tgt_mask = tgt_mask))

# test build_model
# model = build_model()
# batch = next(iter(train_set))
# src, tgt, src_mask, tgt_mask = batch
# print(type(src),src.shape)
# print(summary(model,src = src,tgt = tgt,src_mask = src_mask,tgt_mask = tgt_mask))
# outputs = model(src,tgt,src_mask,tgt_mask)
# print(outputs.shape)

training and validation process
======
Noam optimizer
------

In [16]:
# reference : https://nlp.seas.harvard.edu/2018/04/03/attention.html
class NoamOpt:
    def __init__(self,dictionary_length,factor,warmup,optimizer):
        self.dict_len = dictionary_length
        self.factor = factor
        self.warmup = warmup
        self.optimizer = optimizer
        self._step = 0
        self._rate = 0
    def step(self):
        self._step += 1
        self._rate = self.factor *(self.dict_len ** (-0.5) * \
        min(self._step ** (-0.5), self._step * self.warmup ** (-1.5)))

        self.optimizer.param_groups[0]["lr"] = self._rate
        self.optimizer.step()
    def zero_grad(self):
        return self.optimizer.zero_grad()

    def state_dict(self):
        return self.optimizer.state_dict()

    def load_state_dict(self,state_dict):
        return self.optimizer.load_state_dict(state_dict)

    def set_step(self,step):
        self._step = step
# test NoamOpt:
# x = torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9)
# x.param_groups[0]["lr"]

label smoothing
------

In [17]:
import torch
import torch.nn as nn
class LabelSmoothedCrossEntropyCriterion(nn.Module):
  def __init__(self,batch_size,dictionary_length,padding_id,smoothing,apply_mask):
        super().__init__()
        self.dict_len = dictionary_length
        self.smoothing = smoothing
        self.padding_id = padding_id
        shift = torch.full(size = (batch_size,1), dtype = torch.long, fill_value = self.padding_id)
        self.register_buffer("shift",shift)
        self.apply_mask = apply_mask
  def forward(self, is_last_batch, outputs , label):

    # step1 : when using label in validation, shift is needed.
    # label_shift : {type : tensor , shape : batch  X (max_sentence_length-1)
    # value : int}
    label_shift = label[:,1:]
    # shift : {type : tensor , shape : batch  X 1 ,value : self.padding_id}

    # label_shift : {type : tensor , shape : batch  X max_sentence_length
    # value : int}
    if is_last_batch:
      label_shift = torch.cat((label_shift,self.shift[:label.size(0),:]),dim = 1)

    else:
      label_shift = torch.cat((label_shift,self.shift),dim = 1)

    # step2 : convert label to onehot tensor, then apply label smoothing
    # label_onehot : {type : tensor , shape : batch  X max_sentence_length X dictionary_length
    # value : 0 or 1}
    label_onehot = F.one_hot(label_shift,self.dict_len).float()
    # add : {type : float}
    add = self.smoothing / (self.dict_len)
    # label_onehot : {type : tensor , shape : batch  X max_sentence_length X dictionary_length
    # value : add or 1+add}
    label_onehot += add
    # label_smoothed : {type : tensor , shape : batch  X max_sentence_length X dictionary_length
    # value : add or 1+add-self.smoothing}
    label_smoothed = label_onehot.masked_fill_((label_onehot > 1),float(1-self.smoothing+add))

    '''
    Question : Is padding really needed?

      If padding is applied, the model can't learning that there should be
    no other interger but padd_id after the first bos_id.

      But if padding is ignored at the beginning of the training process,
    it's difficult to train the front part. The reason may be that loss of
    the padding part is much easier to reduce then the front part , so the
    parameters take a local minimun whose padding part is good but the
    front is not good enough rapidly and keeping inside it.

    So we start with padding, and remove padding when fine-tune model.
    '''
    # step3 : use padding mask to ignore to loss from padding id, then calculate loss.
    # loss : {type : tensor , shape : batch  X max_sentence_length X 1, value : float}
    loss = -1*torch.sum((outputs*label_smoothed),dim = -1)
    if self.apply_mask == True:
      # label_padding_mask {type : tensor , shape : batch  X max_sentence_length, value : bool}
      label_padding_mask = (label == self.padding_id)
      # loss : {type : tensor , shape : batch  X max_sentence_length,
      # value : 0 or add or 1+add-self.smoothing}
      loss = loss.masked_fill_(label_padding_mask,0)
    # # ignore_index_number : {type : int}
    # ignore_index_number = (mask_loss == 0).sum().item()
    # avg_loss : {type : int}
    # avg_loss = mask_loss.sum()/(mask_loss.size(0)*mask_loss.size(1)-ignore_index_number)
    avg_loss = loss.sum()/loss.size(0)
    return(avg_loss)

# test LabelSmoothedCrossEntropyCriterion
# cal1 = LabelSmoothedCrossEntropyCriterion()
# print(cal1(outputs,tgt))

# ignore_index not work correctly
# def LabelSmoothedCrossEntropy(outputs , label,dictionary_length,smooth,padding_id):
#   print(outputs.shape)
#   print(label.shape)
#   label_onehot = label.transpose(-1,-2).squeeze()
#   outputs = outputs.transpose(-1,-2)
#   cal_loss = nn.CrossEntropyLoss(ignore_index = padding_idx,reduction = "mean", label_smoothing=smooth)
#   return cal_loss(outputs,label_onehot)

In [18]:
# see https://arxiv.org/pdf/1512.00567.pdf page 7

#Ref 1 : Hong-Yi Li ML2021 HW5

# class LabelSmoothedCrossEntropyCriterion(nn.Module):
#     def __init__(self, smoothing, ignore_index=None, reduce=True):
#         super().__init__()
#         self.smoothing = smoothing
#         self.ignore_index = ignore_index
#         self.reduce = reduce

#     def forward(self, lprobs, target):
#         if target.dim() == lprobs.dim() - 1:
#             target = target.unsqueeze(-1)
#         # nll: Negative log likelihood，the cross-entropy when target is one-hot. following line is same as F.nll_loss
#         nll_loss = -lprobs.gather(dim=-1, index=target)
#         #  reserve some probability for other labels. thus when calculating cross-entropy,
#         # equivalent to summing the log probs of all labels
#         smooth_loss = -lprobs.sum(dim=-1, keepdim=True)
#         if self.ignore_index is not None:
#             pad_mask = target.eq(self.ignore_index)
#             nll_loss.masked_fill_(pad_mask, 0.0)
#             smooth_loss.masked_fill_(pad_mask, 0.0)
#         else:
#             nll_loss = nll_loss.squeeze(-1)
#             smooth_loss = smooth_loss.squeeze(-1)
#         if self.reduce:
#             nll_loss = nll_loss.sum()
#             smooth_loss = smooth_loss.sum()
#         # when calculating cross-entropy, add the loss of other labels
#         eps_i = self.smoothing / lprobs.size(-1)
#         loss = (1.0 - self.smoothing) * nll_loss + eps_i * smooth_loss
#         return loss

#Ref 2 : By hemingkx : https://github.com/hemingkx/ChineseNMT

# class LabelSmoothing(nn.Module):
#     """Implement label smoothing."""

#     def __init__(self, size, padding_idx, smoothing=0.0):
#         super(LabelSmoothing, self).__init__()
#         self.criterion = nn.KLDivLoss(size_average=False)
#         self.padding_idx = padding_idx
#         self.confidence = 1.0 - smoothing
#         self.smoothing = smoothing
#         self.size = size
#         self.true_dist = None


#     def forward(self, x, target):
#         assert x.size(1) == self.size
#         true_dist = x.data.clone()
#         true_dist.fill_(self.smoothing / (self.size - 2))
#         true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
#         true_dist[:, self.padding_idx] = 0
#         mask = torch.nonzero(target.data == self.padding_idx)
#         if mask.dim() > 0:
#             true_dist.index_fill_(0, mask.squeeze(), 0.0)
#         self.true_dist = true_dist
#         return self.criterion(x, Variable(true_dist, requires_grad=False))

beam search
------

In [19]:
import time
from tqdm import tqdm
class Decode_With_Beam_Search(nn.Module):
    def __init__(self,batch_size,model,beam_num,max_sentence_length,
           dictionary_length,bos_id,padding_id):
      super().__init__()
      self.batch_size = batch_size
      self.model = model
      self.beam_num = beam_num
      self.max_sentence_length = max_sentence_length
      self.dictionary_length = dictionary_length
      self.bos_id = bos_id
      self.padding_id = padding_id
      # decoder_input : {type : tensor , shape : Batch X 1 , value : bos_id}
      decoder_input = torch.full(size = (self.batch_size,1),fill_value = self.bos_id)
      self.register_buffer("decoder_input",decoder_input)
      # repeat : {type : tensor , shape : Batch ,value : beam_num}
      # each row repeat beam_num times before concatenate
      repeat = torch.full([self.batch_size],fill_value = self.beam_num)
      self.register_buffer("repeat",repeat)
      # decoder_probability {type : tensor , shape : Batch X beam_num X 1, value : 0}
      decoder_probability = torch.full(size = (self.batch_size,self.beam_num,1),fill_value = 0.0)
      self.register_buffer("decoder_probability",decoder_probability)

      # padding : {type : tensor , shape : (Batch X beam_num) X (max_sentence_length-(id+1)) ,value : int}
      padding = torch.full(size = (batch_size*self.beam_num,self.max_sentence_length),fill_value = self.padding_id)
      self.register_buffer("padding",padding)

      # row : {type : tensor , shape : batch X 1, value : [[0],[1],[2],...]}
      row = torch.tensor(range(self.batch_size)).unsqueeze(1)
      self.register_buffer("row",row)

    def forward(self,is_last_batch,src,src_mask):
      if is_last_batch:
        batch = src.size(0)
      else :
        batch = self.batch_size

      if self.beam_num > batch:
        beam_num = batch
      else :
        beam_num = self.beam_num

      decoder_input = self.decoder_input[:batch,:]
      repeat = self.repeat[:batch]
      decoder_probability = self.decoder_probability[:batch,:,]
      padding = self.padding[:batch*beam_num,:]
      row = self.row[:batch,:]

      # step 1 prepare first input

      # memory : {type : tensor , shape : Batch X max_sentence_length X encoder_output_dim ,value : arbitary float}
      memory = self.model.encoder(is_last_batch,src,src_mask)
      # padding_init : {type : tensor , shape : (Batch X beam_num) X (max_sentence_length-1) ,value : int}
      padding_init = padding[:batch,:self.max_sentence_length-1]
      # input_with_padding : {type : tensor , shape : (Batch X beam_num) X max_sentence_length,
      # value : [[bos_id],[any_id],...[padding_id],....] X Batch}
      input_with_padding = torch.cat((decoder_input,padding_init),dim = 1)
      # add : {type : tensor , shape : Batch X beam_num X 1,
      # value : [[bos_id],[any_id],...[padding_id],....] X Batch}
      add_prob, add = \
      torch.topk(self.model.decoder(is_last_batch,memory,decoder_input,None)[:,1,:],k = beam_num)
      # add : {type : tensor , shape : (Batch X beam_num) X 1,
      # value : [[bos_id],[any_id],...[padding_id],....] X Batch}
      add = add.view(batch*beam_num,1)
      # decoder_beam_expand : {type : tensor , shape : (Batch X beam_num) X 1 ,value : bos_id}
      decoder_beam_expand = torch.repeat_interleave(decoder_input,repeat,dim=0)
      # decoder_beam_expand : {type : tensor , shape : (Batch X beam_num) X 2 ,value : int}
      decoder_beam_expand = torch.cat((decoder_beam_expand,add),dim = 1)

      # decoder_probability : {type : tensor , shape : Batch X beam_num X 1,
      # value : [P1,P2,P3...] X beam_num times (Pk is log_softmax probability)}

      decoder_probability = (decoder_probability+add_prob.unsqueeze(-1))

      # memory_beam_expand : {type : tensor ,
      # shape : (Batch X beam_num) X max_sentence_length X encoder_output_dim ,value : float}
      memory_beam_expand = torch.repeat_interleave(memory,repeat,dim=0)


      # step 2 : iteration to create sentence

      for id in range(self.max_sentence_length-2):
        # padding : {type : tensor , shape : (Batch X beam_num) X (max_sentence_length-(id+2)) ,value : int}

        padding = padding[:,:self.max_sentence_length-(id+2)]
        # out_with_padding : {type : tensor , shape : (Batch X beam_num) X max_sentence_length,
        # value : [[bos_id],[any_id],...[padding_id],....] X Batch}
        out_with_padding = torch.cat((decoder_beam_expand,padding),dim = 1)

        # model decoder spend more than 80% of calculation time
        # out_add : {type : tensor , shape : Batch X beam_num X dictionary_length ,value : int}
        out_add = self.model.decoder(is_last_batch,memory_beam_expand,out_with_padding,None)[:,id+2,:]\
              .view(batch,beam_num,self.dictionary_length)

        # decoder_beam_expand : {type : tensor , shape : (Batch X beam_num) X (id+2) ,value : int}
        # decoder_probability {type : tensor , shape : Batch X beam_num X 1 , value : log_softmax probability}

        decoder_beam_expand , decoder_probability = \
        self.beam_search_one_step(batch,beam_num,repeat,row,decoder_beam_expand,decoder_probability,out_add)

      # step 3 : choose the best sentences

      # decoder_beam_expand : {type : tensor , shape : Batch X beam_num X (max_sentence_length) ,value : 0 or 1}
      decoder_beam_expand = decoder_beam_expand.view(batch,beam_num,self.max_sentence_length)
      # decoder_out : {type : tensor , shape :  Batch X max_sentence_length ,
      # value : [[int,int,...],[int,int...],...]}
      decoder_out =  decoder_beam_expand[:,0,:].view(batch,-1)

      return decoder_out,F.one_hot(decoder_out,self.dictionary_length).float()

    def beam_search_one_step(self,batch,beam_num,repeat,row,sentences,p_sentences,n_beam_output):
    # sentences : {type : tensor , shape : (batch X beam_num) X now_sentences_length X 1 value : int}
    # p_sentences : {type : tensor , shape : batch X beam_num X 1 value : log_softmax probability}
    # n_beam_output : {type : tensor , shape : batch X beam_num X dictionary_length,
    # value : [P1,P2,P3...] X beam_num times (Pk in [0,1])}
      '''
      TO DO : (set beam num = K)
      for every batch:
      expand sentences(total number = K) K times (so there are K-square sentences),then concat with
      the index of top K consequence of each beam(total K beams) in n_beam_output (so there are also K-square values).
      '''

      # sentences : {type : tensor , shape : batch X beam_num X now_sentences_length value : int}
      sentences = sentences.view(batch,beam_num,-1)

      # repeat : {type : tensor , shape : beam_num ,value : beam_num}
      # each row repeat beam_num times before concatenate
      repeat = repeat[:beam_num]
      # sentences_expand : {type : tensor , shape : batch X (beam_num X beam_num) X now_sentences_length ,
      # value : [[[A,B...] X beam_num times,[C,D...] X beam_num times}...] A,B,C,D...are int}
      sentences_expand = torch.repeat_interleave(sentences,repeat,dim=1)
      # sentences_expand = sentences.repeat(1,beam_num,1topk_)


      # topk_prob : {type : tensor , shape : batch X beam_num X beam_num, value : log_softmax probability}
      # topk_index : {type : tensor , shape : batch X beam_num X beam_num, value : int}
      topk_prob, topk_index = torch.topk(n_beam_output,dim = -1,k = beam_num)
      # topk_index : {type : tensor , shape : batch X (beam_num X beam_num) X 1, value : int}
      topk_index = topk_index.view(batch,-1,1)

      # sentences : {type : tensor , shape : batch X (beam_num X beam_num) X (now_sentences_length+1), value : int}
      sentences_expand = torch.cat((sentences_expand,topk_index),dim = -1)

      '''
      TO DO :
      multipies p_sentences with the probability of top K consequence of each beam(total K beams) in n_beam_output
      (so there are also K-square values).

      The final step is to choose Top K consequence from K-square sentences by using p_sentences.
      '''

      # p_sentences : {type : tensor , shape : batch X (beam_num X beam_num),
      # value : [P1,P2,P3...] X beam_num times (Pk is log_softmax probability)}
      p_sentences = (p_sentences+topk_prob).view(batch,-1)
      # p_sentences : {type : tensor , shape : batch X beam_num, value : log_softmax probability}
      # p_index : {type : tensor , shape : batch X beam_num, value : int}
      p_sentences, p_index = torch.topk(p_sentences, dim = 1, k = beam_num)
      p_sentences = p_sentences.unsqueeze(-1)
      # row : {type : tensor , shape : batch X 1, value : [[0],[1],[2],...]}
      # sentences : {type : tensor , shape : batch X beam_num X (now_sentences_length+1), value : log_softmax probability}
      new_sentences = sentences_expand[row, p_index].view(batch*beam_num,-1)

      return new_sentences,p_sentences
# test Decode_With_Beam_Search
# decode_model = Decode_With_Beam_Search(batch_size = 5,
#                     model = model,
#                     beam_num = 2,
#                     max_sentence_length = 12,
#                     dictionary_length = 15,
#                     bos_id = 2,
#                     padding_id = 0)
# output = decode_model(False,src,src_mask)

# test repeat and sentence select

# x = torch.randint(0,10,(2,5))
# print(x)
# r1 = x.repeat(3,1).view(3,2,-1)
# print(r1)
# r2 = x.repeat_interleave(torch.tensor([3,3]),dim=0).view(3,2,-1)
# print(r2)
# row = torch.tensor(range(3)).unsqueeze(1)
# p_index = torch.tensor([[1,0],[0,1],[0,1]])
# new_sentences = r1[row, p_index].view(6,-1)
# print(new_sentences)

# test decode_with_beam_search

bleu
------

In [20]:
import numpy as np
from torcheval.metrics.functional.text import bleu
def get_bleu_score(outputs,tgt,tgt_tokenizer,eos_id):
    outputs = np.array(outputs.detach().tolist())
    outputs = [x[:np.nonzero(x == eos_id)[0][0]].tolist() if len(np.nonzero(x == eos_id)[0])>0 \
              else x.tolist() for x in outputs ]

    outputs_decode = tgt_tokenizer.decode(outputs)
    out = [outputs_decode[i] for i in range(len(outputs_decode)) if len(outputs_decode[i])>= 4]
    out = [" ".join(list(x)) for x in out]
    tgt_decode = tgt_tokenizer.decode(tgt.detach().tolist())
    tgt = [tgt_decode[i] for i in range(len(outputs_decode)) if len(outputs_decode[i])>= 4]
    tgt = [" ".join(list(x)) for x in tgt]
    return bleu.bleu_score(out, tgt, n_gram=4).detach().item()
# test bleu
# test_tokenizer = spm.SentencePieceProcessor(model_file = "/content/spm_8000_zh.model")
# candidates = torch.tensor([[21,3,9,99,42],[5,78,89,3,31]])
# references = torch.tensor([[18,5,9,3,42],[3,5,78,89,50]])
# get_bleu_score(candidates,references,test_tokenizer,3)

train and validation function
------

In [21]:
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.optim import AdamW

def train_one_batch(device,model,loss_calculator,is_last_batch,
          src,tgt,src_mask,tgt_mask,dictionary_length,
          optimizer):

    outputs = model(is_last_batch,src,tgt,src_mask,tgt_mask)

    loss = loss_calculator(is_last_batch,outputs,tgt)
    loss.backward()

    optimizer.step()
    optimizer.zero_grad()
    return loss.detach().item(),outputs[0].detach()

def valid(device,model,loss_calculator,batch_size_setting,valid_loader,beam_num,max_sentence_length,
      dictionary_length,bos_id,eos_id,pad_id,tgt_tokenizer):
    batch_loss = []
    batch_bleu_score = []
    with torch.no_grad():
      for val_batch in tqdm(valid_loader,desc="valid_step", unit=" step"):
        src,tgt,src_mask,tgt_mask = val_batch
        src,tgt,src_mask = src.to(device),tgt.to(device),src_mask.to(device)

        batch_size = src.size(0)

        is_last_batch = False
        if batch_size != batch_size_setting:
          is_last_batch = True
        decode_model = Decode_With_Beam_Search(batch_size,model,beam_num,max_sentence_length,
                            dictionary_length,bos_id,pad_id)
        decode_model.to(device)
        outputs_in_word,outputs = decode_model(is_last_batch,src,src_mask)
        # outputs_in_word,outputs = decode_with_beam_search(device,is_last_batch,model,src,src_mask,beam_num,\
        #       max_sentence_length,dictionary_length,bos_id,pad_id)


        loss = loss_calculator(is_last_batch,outputs,tgt)

        bleu_score = get_bleu_score(outputs_in_word,tgt,tgt_tokenizer,eos_id)

        batch_loss.append(loss)
        batch_bleu_score.append(bleu_score)

      avg_valid_loss = batch_loss.sum()/len(batch_loss).detach().item()
      avg_bleu_score = batch_bleu_score.sum()/len(batch_bleu_score).detach().item()

    return avg_valid_loss,avg_bleu_score

main function
------

In [31]:
from tqdm import tqdm
def main(setting,dataset_is_prepare = False,load_model = False,fine_tune = False):

    # set random seed
    myseed = 1
    np.random.seed(myseed)
    torch.manual_seed(myseed)
    if torch.cuda.is_available():
      torch.cuda.manual_seed_all(myseed)

    # data set & tokenizer
    if not dataset_is_prepare:
        clean_data_and_save(
        path_doc = setting["data_info"]["document"],
        raw_src_path = setting["data_info"]["source"]["raw_data_path"],
        raw_tgt_path = setting["data_info"]["target"]["raw_data_path"],
        clean_src_path = setting["data_info"]["source"]["clean_data_path"],
        clean_tgt_path = setting["data_info"]["target"]["clean_data_path"],
        threshold = setting["tokenized_setting"]["max_l"])

        src_tokenizer,tgt_tokenizer = tokenized_data(
            vocab_size = setting["tokenized_setting"]["vocab_size"],
            tokenized_setting = {k:setting["tokenized_setting"][k] for k in \
                      set(list(setting["tokenized_setting"].keys()))-{"vocab_size","max_l"}},
            max_l = setting["tokenized_setting"]["max_l"],
            path_doc = setting["data_info"]["document"],
            clean_src_path = setting["data_info"]["source"]["clean_data_path"],
            clean_tgt_path = setting["data_info"]["target"]["clean_data_path"],
            src_lang = setting["data_info"]["source"]["lang"],
            tgt_lang = setting["data_info"]["target"]["lang"],
            st_train_path = setting["data_info"]["source"]["tokenized_train_data"],
            st_val_path = setting["data_info"]["source"]["tokenized_val_data"],
            tt_train_path = setting["data_info"]["target"]["tokenized_train_data"],
            tt_val_path = setting["data_info"]["target"]["tokenized_val_data"],
            bos_id = setting["tokenized_setting"]["bos_id"],
            eos_id = setting["tokenized_setting"]["eos_id"],
            pad_id = setting["tokenized_setting"]["pad_id"])
    else:
        src_tokenizer,tgt_tokenizer = get_tokenizers(
            path_doc = setting["data_info"]["document"],
            vocab_size = setting["tokenized_setting"]["vocab_size"],
            src_lang = setting["data_info"]["source"]["lang"],
            tgt_lang = setting["data_info"]["target"]["lang"],)

    # data loader
    train_batch_size_setting = setting["training_hparas"]["train_batch_size"]
    valid_batch_size_setting = setting["training_hparas"]["valid_batch_size"]
    train_loader,valid_loader = get_data_set(
        train_batch_size = train_batch_size_setting,
        valid_batch_size = valid_batch_size_setting,
        num_workers = setting["training_hparas"]["workers"],
        path_doc = setting["data_info"]["document"],
        st_train_path = setting["data_info"]["source"]["tokenized_train_data"],
        st_val_path = setting["data_info"]["source"]["tokenized_val_data"],
        tt_train_path = setting["data_info"]["target"]["tokenized_train_data"],
        tt_val_path = setting["data_info"]["target"]["tokenized_val_data"],
        pad_id = setting["tokenized_setting"]["pad_id"])
    train_iter = iter(train_loader)
    valid_iter = iter(valid_loader)

    # model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model =  build_model(
          max_sentence_length = setting["tokenized_setting"]["max_l"],
          dictionary_length = setting["tokenized_setting"]["vocab_size"],
          padding_idx = setting["tokenized_setting"]["pad_id"],
          encoder_embedding_dimension = setting["model"]["encoder_embedding_dimension"],
          decoder_embedding_dimension = setting["model"]["decoder_embedding_dimension"],
          feedforward_dimension = setting["model"]["feedforward_dimension"],
          num_heads = setting["model"]["num_heads"],
          dropout_p = setting["model"]["dropout_p"],
          layer_num = setting["model"]["layer_num"])

    if load_model:
      checkpoint = torch.load(setting["training_hparas"]["model_temporary_saving_path"])
      model.load_state_dict(checkpoint['model_state_dict'])

    model = model.to(device)

    apply_mask = True
    if fine_tune:
     apply_mask = False

    train_loss_calculator = LabelSmoothedCrossEntropyCriterion(
                batch_size = train_batch_size_setting,
                dictionary_length = setting["tokenized_setting"]["vocab_size"],
                padding_id = setting["tokenized_setting"]["pad_id"],
                smoothing = setting["training_hparas"]["label_smoothing"],
                apply_mask = apply_mask)

    valid_loss_calculator = LabelSmoothedCrossEntropyCriterion(
            batch_size = valid_batch_size_setting,
            dictionary_length = setting["tokenized_setting"]["vocab_size"],
            padding_id = setting["tokenized_setting"]["pad_id"],
            smoothing = 0,
            apply_mask = apply_mask)

    train_loss_calculator,valid_loss_calculator = \
    train_loss_calculator.to(device),valid_loss_calculator.to(device)

    # optimizer
    optimizer = torch.optim.Adam(model.parameters(), **(setting["training_hparas"]["optimization"]["optimizer"]))

    Noam_optimizer = NoamOpt(
             dictionary_length = setting["tokenized_setting"]["vocab_size"],
             factor = setting["training_hparas"]["optimization"]["factor"],
             warmup = setting["training_hparas"]["optimization"]["warmup"],
             optimizer = optimizer)
    if load_model:
      Noam_optimizer.set_step(checkpoint['step'])
      Noam_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    # step
    step = 0
    if load_model:
      step = checkpoint['step']+1
    print(f"Start step is {step}")
    total_step = setting["training_hparas"]["total_step"]
    early_stop_epoch = setting["training_hparas"]["early_stop_step"]
    do_valid_steps = setting["training_hparas"]["do_valid_step"]
    early_stop_count = 0
    progress_bar = tqdm(total = do_valid_steps-step, desc="train_step", unit=" step")

    # output datas
    train_loss_every_batchs = []
    valid_loss = []
    bleu_score = []
    best_bleu_score = 0

    while step <= total_step:

      # training
      # iter batch
      try:
        train_batch = next(train_iter)
      except StopIteration:
        train_iter = iter(train_loader)
        train_batch = next(train_iter)

      # compute batch loss and update parameters in model
      model.train()

      src,tgt,src_mask,tgt_mask = train_batch
      src,tgt,src_mask,tgt_mask = src.to(device),tgt.to(device),\
                     src_mask.to(device),tgt_mask.to(device)
      batch_size = src.size(0)

      is_last_batch = False
      if batch_size != train_batch_size_setting:
        is_last_batch = True

      train_loss, test_sentence = train_one_batch(
              device = device,
              model = model,
              loss_calculator = train_loss_calculator,
              is_last_batch = is_last_batch,
              src = src,
              tgt = tgt,
              src_mask = src_mask,
              tgt_mask = tgt_mask,
              dictionary_length = setting["tokenized_setting"]["vocab_size"],
              optimizer = Noam_optimizer)

      train_loss_every_batchs.append(train_loss)
      temp_save_and_print = setting["training_hparas"]["temp_save_step"]
      if (step+1) % (temp_save_and_print) == 0:
        loss_list = train_loss_every_batchs[int(-1*(temp_save_and_print)):]
        print(sum(loss_list) / len(loss_list))
        print(tgt_tokenizer.decode(torch.argmax(test_sentence,dim = -1).tolist()))
        print(tgt_tokenizer.decode(tgt[0].detach().tolist()))
        torch.save({'model_state_dict': model.state_dict(),
            'optimizer_state_dict': Noam_optimizer.state_dict(),
            'step': step},
            setting["training_hparas"]["model_temporary_saving_path"])

      progress_bar.update()
      if (step+1) % do_valid_steps == 0:

        print(train_loss_every_batchs[-1])

        progress_bar.close()

        model.eval()
        avg_val_loss,avg_bleu_score = valid(
                        device = device,
                        model = model,
                        loss_calculator = valid_loss_calculator,
                        batch_size_setting = valid_batch_size_setting,
                        valid_loader = valid_loader,
                        beam_num = setting["training_hparas"]["beam_num"],
                        max_sentence_length = setting["tokenized_setting"]["max_l"],
                        dictionary_length = setting["tokenized_setting"]["vocab_size"],
                        bos_id = setting["tokenized_setting"]["bos_id"],
                        eos_id = setting["tokenized_setting"]["eos_id"],
                        pad_id = setting["tokenized_setting"]["pad_id"],
                        tgt_tokenizer = tgt_tokenizer)
        valid_loss.append(avg_val_loss)
        bleu_score.append(avg_bleu_score)

        # print avg loss
        print(f"average train loss = {sum(train_loss_every_batchs[-1*do_valid_steps:-1])/len(do_valid_steps):.4f}")
        print(f"average valid loss = {valid_loss[-1]:.4f}")
        print(f"average valid loss = {bleu_score[-1]:.4f}")

        # saving model and check early stop criterion
        if bleu_score[-1] > best_bleu_score:
          torch.save(model.state_dict(), setting["training_hparas"]["model_saving_path"])
        else :
          early_stop_count += 1

        if early_stop_count == early_stop_epoch:
          break

        progress_bar = tqdm(total = do_valid_steps, desc="train_step", unit=" step")

      step += 1
    progress_bar.close()

    return train_loss_every_batchs,valid_loss,bleu_score

In [ ]:
main(setting, dataset_is_prepare = True, load_model = True)
# gc.collect()

100%|██████████| 3879/3879 [00:00<00:00, 15920.38it/s]


Now step is 40000


train_step:   5%|▍         | 999/20000 [15:31<4:55:20,  1.07 step/s]

86.41491871643066
活在在一個觀的中一個。的,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,著著著著著著,著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著著
生命結構就在外星空下形成了。


train_step:  10%|▉         | 1999/20000 [31:06<4:38:30,  1.08 step/s]

...

train_step:  55%|█████▍    | 10999/20000 [2:51:11<2:19:26,  1.08 step/s]

81.38089778900147
我們不感謝祝,沒有在在三00年前就的貓蜓就貓王。。。。。的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而而
而且我們要慶幸自己不是生活在3億年前,那時的蜻蜓跟貓一樣大。


train_step:  60%|█████▉    | 11999/20000 [3:06:43<2:03:57,  1.08 step/s]

84.9609408493042
這點嚇。。。。。。。。。,,,,,,,,,不我,而,,不而而,,不不,,而,,,,,,,,,,,我我我,我我我我,,我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我我
有點可怕


train_step:  65%|██████▍   | 12924/20000 [3:21:05<1:50:03,  1.07 step/s]

get plot
------

In [24]:
import matplotlib as plt
class get_plot():
    def __init__(self,train_loss_every_batchs,valid_loss,bleu_score,
           step = setting["training_hparas"]["do_valid_step"]):
      self.train = train_loss_every_batchs
      self.val = valid_loss
      self.score = bleu_score
      self.step = step
    def plot_train_loss(self,figsize = [6,12],ylims = [0,10]):
      index = [i for i in range(len(self.train))]
      plt.figure(figsize = figsize, layout='constrained')
      plt.plot(data = [index, self.train],
           color = "ted:blue", label = "train loss")
      plt.ylim(ylims)
      plt.xlabel("step")
      plt.ylabel("loss")
      plt.legend()
      plt.show()
    def plot_loss(self,figsize = [6,12],ylims = [0,10]):
      avg_step = min(self.step//10,1)
      train_index = list(range(0,len(self.train),self.step))

      train_avg = [self.train[:avg_step].mean()] + \
             [self.train[i-avg_step:i+avg_step].mean() for i in train_index[1:-1]] + \
             [self.train[-avg_step:].mean()]

      plt.figure(figsize = figsize, layout='constrained')
      plt.plot(data = [train_index, train_avg],
          color = "ted:blue", label = "train loss")
      plt.plot(data = [train_index, self.val],
          color = "ted:red", label = "val loss")
      plt.ylim(ylims)
      plt.xlabel("step")
      plt.ylabel("loss")
      plt.legend()
      plt.show()
    def plot_bleu_score(self,figsize = [6,12],ylims = [0,10]):
      train_index = list(range(0,len(self.train),self.step))


      plt.figure(figsize = figsize, layout='constrained')
      plt.plot(data = [train_index, self.score],
          color = "ted:blue", label = "train loss")

      plt.ylim(ylims)
      plt.xlabel("step")
      plt.ylabel("bleu_score")
      plt.legend()
      plt.show()

Part 4 inference
======
infer dataset
------

In [25]:
import torch
from tqdm import tqdm
import numpy as np
from torch.utils.data import Dataset
# make dataset
class infer_dataset(Dataset):
    def __init__(self,list_of_sentences,src_tokenizer,bos_id,eos_id,pad_id,max_l):
      # clean
      infer_list = [s[0] for s in list(map(clean_s_en,list_of_sentences))]
      infer_list = [" ".join(s) for s in list(map(divide_by_END,infer_list))]
      # padding and tokenized
      padding_src = []
      len_s = 0
      for sentence in infer_list:
        s = src_tokenizer.encode(sentence, out_type=int)
        s = np.append(s,[eos_id])
        s = np.append([bos_id],np.pad(s,(0, max_l-len(s)-1), constant_values = pad_id))
        padding_src.append(s.tolist())
      # convert to tensor
      self.infer_set = torch.LongTensor(padding_src)
    def __getitem__(self, index):
      return self.infer_set[index]

    def __len__(self):
      return len(self.infer_set)

    @classmethod
    def padding_mask_batch(cls,batch,pad_id):
      """Collate a batch of data."""
      infer = torch.stack(batch)
      infer_padding = (infer == pad_id)
      return infer, infer_padding

infer function
------

In [26]:
def infer(device,model,batch_size_setting,infer_loader,beam_num,max_sentence_length,
      dictionary_length,bos_id,eos_id,pad_id):

    with torch.no_grad():
      for infer_batch in tqdm(infer_loader,desc="infer_step", unit=" step"):
        infer,infer_mask = infer_batch
        infer,infer_mask = infer.to(device),infer_mask.to(device)

        batch_size = infer.size(0)

        is_last_batch = False
        if batch_size != batch_size_setting:
          is_last_batch = True
        decode_model = Decode_With_Beam_Search(batch_size,model,1,max_sentence_length,
                            dictionary_length,bos_id,pad_id)
        decode_model.to(device)
        outputs_in_word,outputs = decode_model(is_last_batch,infer,infer_mask)



    return outputs_in_word.detach().tolist()

inference main function
------

In [27]:
import json
def inference_main(setting,list_of_sentences):
  # get tokenizer
  src_tokenizer,tgt_tokenizer = get_tokenizers(
            path_doc = setting["data_info"]["document"],
            vocab_size = setting["tokenized_setting"]["vocab_size"],
            src_lang = setting["data_info"]["source"]["lang"],
            tgt_lang = setting["data_info"]["target"]["lang"],)

  max_l = setting["tokenized_setting"]["max_l"]
  bos_id = setting["tokenized_setting"]["bos_id"]
  eos_id = setting["tokenized_setting"]["eos_id"]
  pad_id = setting["tokenized_setting"]["pad_id"]
  num_workers = setting["training_hparas"]["workers"]
  infer_batch_size_setting = setting["training_hparas"]["valid_batch_size"]
  # make dataloader
  infer_set = infer_dataset(list_of_sentences,src_tokenizer,bos_id,eos_id,pad_id,max_l)
  infer_loader = DataLoader(
    infer_set,
    batch_size = infer_batch_size_setting,
    shuffle = True,
    num_workers = num_workers,
    pin_memory = True,
    collate_fn = lambda x : infer_dataset.padding_mask_batch(x,
                   pad_id = pad_id))

  # set model and device
  max_sentence_length = setting["tokenized_setting"]["max_l"]
  dictionary_length = setting["tokenized_setting"]["vocab_size"]

  model =  build_model(
      max_sentence_length = max_sentence_length,
      dictionary_length = dictionary_length,
      padding_idx = setting["tokenized_setting"]["pad_id"],
      encoder_embedding_dimension = setting["model"]["encoder_embedding_dimension"],
      decoder_embedding_dimension = setting["model"]["decoder_embedding_dimension"],
      feedforward_dimension = setting["model"]["feedforward_dimension"],
      num_heads = setting["model"]["num_heads"],
      dropout_p = setting["model"]["dropout_p"],
      layer_num = setting["model"]["layer_num"])

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  checkpoint = torch.load(setting["training_hparas"]["model_temporary_saving_path"])
  model.load_state_dict(checkpoint['model_state_dict'])
  model.to(device)
  # using model to translation
  model.eval()

  beam_num = setting["training_hparas"]["beam_num"]

  outputs_in_word = infer(
                device = device,
                model = model,
                batch_size_setting = infer_batch_size_setting,
                infer_loader = infer_loader,
                beam_num = beam_num,
                max_sentence_length = max_sentence_length,
                dictionary_length = dictionary_length,
                bos_id = bos_id,
                eos_id = eos_id,
                pad_id = pad_id)
  outputs_in_word = tgt_tokenizer.decode(outputs_in_word)

  # print and save output
  outputs = [(list_of_sentences[i],outputs_in_word[i]) for i in range(len(outputs_in_word))]
  print(outputs)
  save_infer_path = setting["inference_out_path"]
  with open(path_doc + save_infer_path, 'w') as out_f:
      for index,line_pair in enumerate(tqdm(outputs)):
        infer_dict = {"index":index,"original":line_pair[0],"translation":line_pair[1]}
        json.dump(infer_dict, out_f)

In [28]:
inference_main(setting,["Hello, world!","this is a pen"])

infer_step: 100%|██████████| 1/1 [00:08<00:00,  8.56s/ step]


[('Hello, world!', '!!!!!!!!!!!!!的!,,,你!!,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————'), ('this is a pen', ',,。。。。的子,,,,,,,子,,著,,,,子,,,,子,,,,子,,,,子,,子,,,,,,子,,,,,,,,,子,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,———————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————

100%|██████████| 2/2 [00:00<00:00, 10824.01it/s]
